# Calculation of MLB Statistics (wOBA, wRAA, wRC)
This file consists the code to calculate wOBA, wRAA, and wRC manually since FanGraphs for these stats are hidden behind a paywall. Note that these are not qualified batters but instead just batters with at least 1 PA. The reason for this is because we want to see how these batters do on an aggregate basis rather than on an average since we're looking at wOBA, wRAA, and wRC.

## Aggregating MLB Data from StatCast

In [1]:
import pandas as pd
import numpy as np

In [16]:
# Read data
df_2015_2016 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2015-2016.csv")
df_2017_2019 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2017-2019.csv")
df_2020_2022 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2020-2022.csv")
df_2023_2025 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2023-2025.csv")

# Aggregate
df_mlb_stats = pd.concat(
    [df_2015_2016, df_2017_2019, df_2020_2022, df_2023_2025], axis=0
)

# Column name fixing
df_mlb_stats.rename(
    columns={
        "last_name, first_name": "Name",
        "player_id": "StatCast_ID",
        "year": "Year",
        "player_age": "Age",
        "pa": "PA",
        "single": "1B",
        "double": "2B",
        "triple": "3B",
        "home_run": "HR",
        "walk": "BB",
        "k_percent": "K%",
        "bb_percent": "BB%",
        "batting_avg": "BA",
        "slg_percent": "SLG",
        "on_base_percent": "OBP",
        "on_base_plus_slg": "OPS",
        "isolated_power": "ISO",
        "babip": "BABIP",
        "b_intent_walk": "IBB",
        "b_sac_fly": "SF",
        "woba": "wOBA",
    },
    inplace=True,
)


# Fix name format from "last_name, first_name" to first name last name"
def fix_name(name: str) -> str:
    last, first = [part.strip() for part in name.split(",", 1)]
    return f"{first} {last}"


df_mlb_stats["Name"] = df_mlb_stats["Name"].apply(fix_name)

# Display
df_mlb_stats

,Name,StatCast_ID,Year,Age,PA,1B,2B,3B,HR,BB,...,BB%,BA,SLG,OBP,OPS,ISO,BABIP,IBB,SF,wOBA
0,Mike Moustakas,519058,2015,26,614,99,34,1,22,43,...,7.0,0.284,0.470,0.348,0.818,0.186,0.294,1,5,0.353
1,Chad Pinder,640461,2016,24,55,7,4,0,1,3,...,5.5,0.235,0.373,0.273,0.646,0.138,0.297,0,1,0.276
2,Manny Piña,444489,2016,29,81,12,4,0,2,10,...,12.3,0.254,0.394,0.346,0.740,0.140,0.296,0,0,0.326
3,Jhonny Peralta,425509,2016,34,313,49,17,1,8,20,...,6.4,0.260,0.408,0.307,0.715,0.148,0.294,0,3,0.308
4,Zack Cozart,446359,2016,30,508,71,28,2,16,37,...,7.3,0.252,0.425,0.308,0.733,0.173,0.274,3,4,0.312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,Dominic Canzone,686527,2025,27,232,43,10,0,7,18,...,7.8,0.287,0.435,0.349,0.784,0.148,0.344,0,2,0.342
1592,Yoán Moncada,660162,2025,30,271,30,12,2,12,31,...,11.4,0.240,0.464,0.343,0.807,0.224,0.288,1,1,0.349
1593,James McCann,543510,2023,33,226,26,14,0,6,9,...,4.0,0.222,0.377,0.269,0.646,0.155,0.274,0,2,0.279
1594,Esteury Ruiz,665923,2023,24,497,84,24,1,5,20,...,4.0,0.254,0.345,0.309,0.654,0.091,0.315,1,1,0.290


## Get annual league wOBA values
Obtained from [FanGraphs](https://www.fangraphs.com/tools/guts?type=cn).

In [18]:
# wOBA scaling from 2015 to 2025 as of Sept 16th 2025
woba_values = {
    "Season": [2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015],
    "wOBA": [
        0.314,
        0.310,
        0.318,
        0.310,
        0.314,
        0.320,
        0.320,
        0.315,
        0.321,
        0.318,
        0.313,
    ],
    "wOBAScale": [
        1.230,
        1.242,
        1.204,
        1.259,
        1.209,
        1.185,
        1.157,
        1.226,
        1.185,
        1.212,
        1.251,
    ],
    "wBB": [
        0.693,
        0.689,
        0.696,
        0.689,
        0.692,
        0.699,
        0.690,
        0.690,
        0.693,
        0.691,
        0.687,
    ],
    "wHBP": [
        0.724,
        0.720,
        0.726,
        0.720,
        0.722,
        0.728,
        0.719,
        0.720,
        0.723,
        0.721,
        0.718,
    ],
    "w1B": [
        0.883,
        0.882,
        0.883,
        0.884,
        0.879,
        0.883,
        0.870,
        0.880,
        0.877,
        0.878,
        0.881,
    ],
    "w2B": [
        1.252,
        1.254,
        1.244,
        1.261,
        1.242,
        1.238,
        1.217,
        1.247,
        1.232,
        1.242,
        1.256,
    ],
    "w3B": [
        1.584,
        1.590,
        1.569,
        1.601,
        1.568,
        1.558,
        1.529,
        1.578,
        1.552,
        1.569,
        1.594,
    ],
    "wHR": [
        2.035,
        2.050,
        2.004,
        2.072,
        2.007,
        1.979,
        1.940,
        2.031,
        1.980,
        2.015,
        2.065,
    ],
    "runSB": [0.200] * 11,
    "runCS": [
        -0.411,
        -0.405,
        -0.422,
        -0.397,
        -0.419,
        -0.435,
        -0.435,
        -0.407,
        -0.423,
        -0.410,
        -0.392,
    ],
    "R/PA": [
        0.119,
        0.117,
        0.122,
        0.114,
        0.121,
        0.125,
        0.126,
        0.117,
        0.122,
        0.118,
        0.112,
    ],
    "R/W": [
        9.808,
        9.683,
        10.028,
        9.524,
        9.973,
        10.282,
        10.296,
        9.714,
        10.048,
        9.778,
        9.421,
    ],
    "cFIP": [
        3.146,
        3.166,
        3.255,
        3.112,
        3.170,
        3.191,
        3.214,
        3.160,
        3.158,
        3.147,
        3.134,
    ],
}

woba_values = pd.DataFrame(woba_values)
woba_values

,Season,wOBA,wOBAScale,wBB,wHBP,w1B,w2B,w3B,wHR,runSB,runCS,R/PA,R/W,cFIP
0,2025,0.314,1.230,0.693,0.724,0.883,1.252,1.584,2.035,0.2,-0.411,0.119,9.808,3.146
1,2024,0.310,1.242,0.689,0.720,0.882,1.254,1.590,2.050,0.2,-0.405,0.117,9.683,3.166
2,2023,0.318,1.204,0.696,0.726,0.883,1.244,1.569,2.004,0.2,-0.422,0.122,10.028,3.255
3,2022,0.310,1.259,0.689,0.720,0.884,1.261,1.601,2.072,0.2,-0.397,0.114,9.524,3.112
4,2021,0.314,1.209,0.692,0.722,0.879,1.242,1.568,2.007,0.2,-0.419,0.121,9.973,3.170
5,2020,0.320,1.185,0.699,0.728,0.883,1.238,1.558,1.979,0.2,-0.435,0.125,10.282,3.191
6,2019,0.320,1.157,0.690,0.719,0.870,1.217,1.529,1.940,0.2,-0.435,0.126,10.296,3.214
7,2018,0.315,1.226,0.690,0.720,0.880,1.247,1.578,2.031,0.2,-0.407,0.117,9.714,3.160
8,2017,0.321,1.185,0.693,0.723,0.877,1.232,1.552,1.980,0.2,-0.423,0.122,10.048,3.158
9,2016,0.318,1.212,0.691,0.721,0.878,1.242,1.569,2.015,0.2,-0.410,0.118,9.778,3.147


## Calculate wRAA

Formula: ((wOBA - wOBA of the entire league) / annual wOBA scale) x PA


In [23]:
def calculate_raa(woba: float, pa: float, year: int) -> float:
    league_woba = woba_values.loc[woba_values["Season"] == year, "wOBA"].iloc[0]
    woba_scale = woba_values.loc[woba_values["Season"] == year, "wOBAScale"].iloc[0]
    return ((woba - league_woba) / woba_scale) * pa


df_mlb_stats["wRAA"] = df_mlb_stats.apply(
    lambda row: calculate_raa(row["wOBA"], row["PA"], row["Year"]), axis=1
)

df_mlb_stats

,Name,StatCast_ID,Year,Age,PA,1B,2B,3B,HR,BB,...,BA,SLG,OBP,OPS,ISO,BABIP,IBB,SF,wOBA,wRAA
0,Mike Moustakas,519058,2015,26,614,99,34,1,22,43,...,0.284,0.470,0.348,0.818,0.186,0.294,1,5,0.353,19.632294
1,Chad Pinder,640461,2016,24,55,7,4,0,1,3,...,0.235,0.373,0.273,0.646,0.138,0.297,0,1,0.276,-1.905941
2,Manny Piña,444489,2016,29,81,12,4,0,2,10,...,0.254,0.394,0.346,0.740,0.140,0.296,0,0,0.326,0.534653
3,Jhonny Peralta,425509,2016,34,313,49,17,1,8,20,...,0.260,0.408,0.307,0.715,0.148,0.294,0,3,0.308,-2.582508
4,Zack Cozart,446359,2016,30,508,71,28,2,16,37,...,0.252,0.425,0.308,0.733,0.173,0.274,3,4,0.312,-2.514851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,Dominic Canzone,686527,2025,27,232,43,10,0,7,18,...,0.287,0.435,0.349,0.784,0.148,0.344,0,2,0.342,5.281301
1592,Yoán Moncada,660162,2025,30,271,30,12,2,12,31,...,0.240,0.464,0.343,0.807,0.224,0.288,1,1,0.349,7.711382
1593,James McCann,543510,2023,33,226,26,14,0,6,9,...,0.222,0.377,0.269,0.646,0.155,0.274,0,2,0.279,-7.320598
1594,Esteury Ruiz,665923,2023,24,497,84,24,1,5,20,...,0.254,0.345,0.309,0.654,0.091,0.315,1,1,0.290,-11.558140
